# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [41]:
variaveis_qualitativas = ['sexo','tipo_renda','educacao','estado_civil','tipo_residencia']

for var in variaveis_qualitativas:
    print(f'\nCategorias para {var}:')
    categorias = df[var].value_counts()
    print(categorias)
    print(f'\nCategorias de {var} ordenadas:')
    ordenadas = categorias.sort_values(ascending=False)
    print(ordenadas)


Categorias para sexo:
sexo
F    10119
M     4881
Name: count, dtype: int64

Categorias de sexo ordenadas:
sexo
F    10119
M     4881
Name: count, dtype: int64

Categorias para tipo_renda:
tipo_renda
Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: count, dtype: int64

Categorias de tipo_renda ordenadas:
tipo_renda
Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: count, dtype: int64

Categorias para educacao:
educacao
Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: count, dtype: int64

Categorias de educacao ordenadas:
educacao
Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: count, dtype: int64

Categorias para estado_civil:
estado_

In [43]:
y, x = patsy.dmatrices('np.log(renda) ~ C(sexo,Treatment(reference="F")) + posse_de_veiculo + C(educacao,Treatment(reference="Secundário")) + posse_de_imovel + qtd_filhos + C(tipo_renda,Treatment(reference="Assalariado")) + educacao + C(estado_civil,Treatment(reference="Casado")) + C(tipo_residencia,Treatment(reference="Casa")) + idade + tempo_emprego + qt_pessoas_residencia', data=df)
x

DesignMatrix with shape (12427, 29)
  Columns:
    ['Intercept',
     'C(sexo, Treatment(reference="F"))[T.M]',
     'posse_de_veiculo[T.True]',
     'C(educacao, Treatment(reference="Secundário"))[T.Primário]',
     'C(educacao, Treatment(reference="Secundário"))[T.Pós graduação]',
     'C(educacao, Treatment(reference="Secundário"))[T.Superior completo]',
     'C(educacao, Treatment(reference="Secundário"))[T.Superior incompleto]',
     'posse_de_imovel[T.True]',
     'C(tipo_renda, Treatment(reference="Assalariado"))[T.Bolsista]',
     'C(tipo_renda, Treatment(reference="Assalariado"))[T.Empresário]',
     'C(tipo_renda, Treatment(reference="Assalariado"))[T.Pensionista]',
     'C(tipo_renda, Treatment(reference="Assalariado"))[T.Servidor público]',
     'educacao[T.Pós graduação]',
     'educacao[T.Secundário]',
     'educacao[T.Superior completo]',
     'educacao[T.Superior incompleto]',
     'C(estado_civil, Treatment(reference="Casado"))[T.Separado]',
     'C(estado_civil, Treat

In [45]:
sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Thu, 29 Aug 2024   Prob (F-statistic):               0.00
Time:                        20:49:42   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 5.1015      0.171     29.790      0.000       4.766       5.437
C(sexo, Treatment(reference="F"))[T.M]                                    0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]                                                  0.0441      0.014      3.119      0.002       0.016       0.072
C(educacao, Treatment(reference="Secundário"))[T.Primário]                1.4391      0.075     19.072      0.000       1.291       1.587
C(educacao, Treatment(reference="Secundário"))[T.Pós graduação]           0.7731      0.068     11.347      0.000       0.640       0.907
C(educacao, Treatment(reference="Secundário"))[T.Superior completo]       0.7664      0.027     28.219      0.000       0.713       0.820
C(educacao, Treatment(reference="Secundário"))[T.Superior incompleto]     0.6980      0.030     23.372      0.000       0.639       0.756
posse_de_imovel[T.True]                                                   0.0829      0.014      5.926      0.000       0.055       0.110
C(tipo_renda, Treatment(reference="Assalariado"))[T.Bolsista]             0.2209      0.241      0.916      0.360      -0.252       0.694
C(tipo_renda, Treatment(reference="Assalariado"))[T.Empresário]           0.1551      0.015     10.387      0.000       0.126       0.184
C(tipo_renda, Treatment(reference="Assalariado"))[T.Pensionista]         -0.3087      0.241     -1.280      0.201      -0.782       0.164
C(tipo_renda, Treatment(reference="Assalariado"))[T.Servidor público]     0.0576      0.022      2.591      0.010       0.014       0.101
educacao[T.Pós graduação]                                                 0.7731      0.068     11.347      0.000       0.640       0.907
educacao[T.Secundário]                                                    1.4249      0.054     26.310      0.000       1.319       1.531
educacao[T.Superior completo]                                             0.7664      0.027     28.219      0.000       0.713       0.820
educacao[T.Superior incompleto]                                           0.6980      0.030     23.372      0.000       0.639       0.756
C(estado_civil, Treatment(reference="Casado"))[T.Separado]                0.3265      0.111      2.935      0.003       0.108       0.545
C(estado_civil, Treatment(reference="Casado"))[T.Solteiro]                0.2672      0.109      2.453      0.014       0.054       0.481
C(estado_civil, Treatment(reference="Casado"))[T.União]                  -0.0343      0.025     -1.361      0.174      -0.084       0.015
C(estado_civil, Treatment(reference="Casado"))[

In [47]:
y, x = patsy.dmatrices('np.log(renda) ~ C(sexo,Treatment(reference="F")) + posse_de_veiculo + C(educacao,Treatment(reference="Secundário")) + posse_de_imovel + qtd_filhos + educacao + C(estado_civil,Treatment(reference="Casado")) + C(tipo_residencia,Treatment(reference="Casa")) + idade + tempo_emprego + qt_pessoas_residencia', data=df)
x

DesignMatrix with shape (12427, 25)
  Columns:
    ['Intercept',
     'C(sexo, Treatment(reference="F"))[T.M]',
     'posse_de_veiculo[T.True]',
     'C(educacao, Treatment(reference="Secundário"))[T.Primário]',
     'C(educacao, Treatment(reference="Secundário"))[T.Pós graduação]',
     'C(educacao, Treatment(reference="Secundário"))[T.Superior completo]',
     'C(educacao, Treatment(reference="Secundário"))[T.Superior incompleto]',
     'posse_de_imovel[T.True]',
     'educacao[T.Pós graduação]',
     'educacao[T.Secundário]',
     'educacao[T.Superior completo]',
     'educacao[T.Superior incompleto]',
     'C(estado_civil, Treatment(reference="Casado"))[T.Separado]',
     'C(estado_civil, Treatment(reference="Casado"))[T.Solteiro]',
     'C(estado_civil, Treatment(reference="Casado"))[T.União]',
     'C(estado_civil, Treatment(reference="Casado"))[T.Viúvo]',
     'C(tipo_residencia, Treatment(reference="Casa"))[T.Aluguel]',
     'C(tipo_residencia, Treatment(reference="Casa"))[T.Co

In [49]:
sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     336.5
Date:                Thu, 29 Aug 2024   Prob (F-statistic):               0.00
Time:                        20:51:00   Log-Likelihood:                -13623.
No. Observations:               12427   AIC:                         2.729e+04
Df Residuals:                   12406   BIC:                         2.744e+04
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 5.1754      0.172     30.119      0.000       4.839       5.512
C(sexo, Treatment(reference="F"))[T.M]                                    0.7790      0.015     53.021      0.000       0.750       0.808
posse_de_veiculo[T.True]                                                  0.0459      0.014      3.232      0.001       0.018       0.074
C(educacao, Treatment(reference="Secundário"))[T.Primário]                1.4417      0.076     19.026      0.000       1.293       1.590
C(educacao, Treatment(reference="Secundário"))[T.Pós graduação]           0.7989      0.068     11.684      0.000       0.665       0.933
C(educacao, Treatment(reference="Secundário"))[T.Superior completo]       0.7814      0.027     28.696      0.000       0.728       0.835
C(educacao, Treatment(reference="Secundário"))[T.Superior incompleto]     0.7115      0.030     23.747      0.000       0.653       0.770
posse_de_imovel[T.True]                                                   0.0839      0.014      5.975      0.000       0.056       0.111
educacao[T.Pós graduação]                                                 0.7989      0.068     11.684      0.000       0.665       0.933
educacao[T.Secundário]                                                    1.4418      0.054     26.521      0.000       1.335       1.548
educacao[T.Superior completo]                                             0.7814      0.027     28.696      0.000       0.728       0.835
educacao[T.Superior incompleto]                                           0.7115      0.030     23.747      0.000       0.653       0.770
C(estado_civil, Treatment(reference="Casado"))[T.Separado]                0.3141      0.112      2.812      0.005       0.095       0.533
C(estado_civil, Treatment(reference="Casado"))[T.Solteiro]                0.2587      0.109      2.365      0.018       0.044       0.473
C(estado_civil, Treatment(reference="Casado"))[T.União]                  -0.0261      0.025     -1.034      0.301      -0.076       0.023
C(estado_civil, Treatment(reference="Casado"))[T.Viúvo]                   0.3446      0.116      2.969      0.003       0.117       0.572
C(tipo_residencia, Treatment(reference="Casa"))[T.Aluguel]                0.0440      0.054      0.810      0.418      -0.063       0.151
C(tipo_residencia, Treatment(reference="Casa"))[T.Com os pais]            0.0062      0.030      0.210      0.834      -0.052       0.064
C(tipo_residencia, Treatment(reference="Casa"))[T.Comunitário]           -0.0990      0.091     -1.084      0.278      -0.278       0.080
C(tipo_residencia, Treatment(reference="Casa"))

In [61]:
y, x = patsy.dmatrices('np.log(renda) ~ C(sexo,Treatment(reference="F")) + posse_de_veiculo + C(educacao,Treatment(reference="Secundário")) + posse_de_imovel + qtd_filhos + educacao + C(estado_civil,Treatment(reference="Casado")) +  idade + tempo_emprego + qt_pessoas_residencia', data=df)
x

DesignMatrix with shape (12427, 20)
  Columns:
    ['Intercept',
     'C(sexo, Treatment(reference="F"))[T.M]',
     'posse_de_veiculo[T.True]',
     'C(educacao, Treatment(reference="Secundário"))[T.Primário]',
     'C(educacao, Treatment(reference="Secundário"))[T.Pós graduação]',
     'C(educacao, Treatment(reference="Secundário"))[T.Superior completo]',
     'C(educacao, Treatment(reference="Secundário"))[T.Superior incompleto]',
     'posse_de_imovel[T.True]',
     'educacao[T.Pós graduação]',
     'educacao[T.Secundário]',
     'educacao[T.Superior completo]',
     'educacao[T.Superior incompleto]',
     'C(estado_civil, Treatment(reference="Casado"))[T.Separado]',
     'C(estado_civil, Treatment(reference="Casado"))[T.Solteiro]',
     'C(estado_civil, Treatment(reference="Casado"))[T.União]',
     'C(estado_civil, Treatment(reference="Casado"))[T.Viúvo]',
     'qtd_filhos',
     'idade',
     'tempo_emprego',
     'qt_pessoas_residencia']
  Terms:
    'Intercept' (column 0)
    

In [63]:
sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.351
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     448.4
Date:                Thu, 29 Aug 2024   Prob (F-statistic):               0.00
Time:                        20:57:36   Log-Likelihood:                -13625.
No. Observations:               12427   AIC:                         2.728e+04
Df Residuals:                   12411   BIC:                         2.740e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 5.1756      0.172     30.138      0.000       4.839       5.512
C(sexo, Treatment(reference="F"))[T.M]                                    0.7809      0.015     53.291      0.000       0.752       0.810
posse_de_veiculo[T.True]                                                  0.0460      0.014      3.241      0.001       0.018       0.074
C(educacao, Treatment(reference="Secundário"))[T.Primário]                1.4375      0.076     19.007      0.000       1.289       1.586
C(educacao, Treatment(reference="Secundário"))[T.Pós graduação]           0.7995      0.068     11.695      0.000       0.666       0.934
C(educacao, Treatment(reference="Secundário"))[T.Superior completo]       0.7823      0.027     28.743      0.000       0.729       0.836
C(educacao, Treatment(reference="Secundário"))[T.Superior incompleto]     0.7124      0.030     23.800      0.000       0.654       0.771
posse_de_imovel[T.True]                                                   0.0836      0.014      6.068      0.000       0.057       0.111
educacao[T.Pós graduação]                                                 0.7995      0.068     11.695      0.000       0.666       0.934
educacao[T.Secundário]                                                    1.4440      0.054     26.577      0.000       1.337       1.550
educacao[T.Superior completo]                                             0.7823      0.027     28.743      0.000       0.729       0.836
educacao[T.Superior incompleto]                                           0.7124      0.030     23.800      0.000       0.654       0.771
C(estado_civil, Treatment(reference="Casado"))[T.Separado]                0.3145      0.112      2.816      0.005       0.096       0.534
C(estado_civil, Treatment(reference="Casado"))[T.Solteiro]                0.2596      0.109      2.374      0.018       0.045       0.474
C(estado_civil, Treatment(reference="Casado"))[T.União]                  -0.0248      0.025     -0.982      0.326      -0.074       0.025
C(estado_civil, Treatment(reference="Casado"))[T.Viúvo]                   0.3432      0.116      2.958      0.003       0.116       0.571
qtd_filhos                                                               -0.2555      0.108     -2.357      0.018      -0.468      -0.043
idade                                                                     0.0051      0.001      6.352      0.000       0.003       0.007
tempo_emprego                                                             0.0610      0.001     59.191      0.000       0.059       0.063
qt_pessoas_residencia                          

In [65]:
y, x = patsy.dmatrices('np.log(renda) ~ C(sexo,Treatment(reference="F")) + posse_de_veiculo + C(educacao,Treatment(reference="Secundário")) + posse_de_imovel + qtd_filhos + educacao + idade + tempo_emprego + qt_pessoas_residencia', data=df)
x

DesignMatrix with shape (12427, 16)
  Columns:
    ['Intercept',
     'C(sexo, Treatment(reference="F"))[T.M]',
     'posse_de_veiculo[T.True]',
     'C(educacao, Treatment(reference="Secundário"))[T.Primário]',
     'C(educacao, Treatment(reference="Secundário"))[T.Pós graduação]',
     'C(educacao, Treatment(reference="Secundário"))[T.Superior completo]',
     'C(educacao, Treatment(reference="Secundário"))[T.Superior incompleto]',
     'posse_de_imovel[T.True]',
     'educacao[T.Pós graduação]',
     'educacao[T.Secundário]',
     'educacao[T.Superior completo]',
     'educacao[T.Superior incompleto]',
     'qtd_filhos',
     'idade',
     'tempo_emprego',
     'qt_pessoas_residencia']
  Terms:
    'Intercept' (column 0)
    'C(sexo, Treatment(reference="F"))' (column 1)
    'posse_de_veiculo' (column 2)
    'C(educacao, Treatment(reference="Secundário"))' (columns 3:7)
    'posse_de_imovel' (column 7)
    'educacao' (columns 8:12)
    'qtd_filhos' (column 12)
    'idade' (column 13

In [67]:
sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.351
Model:                            OLS   Adj. R-squared:                  0.350
Method:                 Least Squares   F-statistic:                     609.9
Date:                Thu, 29 Aug 2024   Prob (F-statistic):               0.00
Time:                        20:58:34   Log-Likelihood:                -13632.
No. Observations:               12427   AIC:                         2.729e+04
Df Residuals:                   12415   BIC:                         2.738e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 5.6040      0.040    139.356      0.000       5.525       5.683
C(sexo, Treatment(reference="F"))[T.M]                                    0.7786      0.015     53.330      0.000       0.750       0.807
posse_de_veiculo[T.True]                                                  0.0463      0.014      3.271      0.001       0.019       0.074
C(educacao, Treatment(reference="Secundário"))[T.Primário]                1.5653      0.059     26.689      0.000       1.450       1.680
C(educacao, Treatment(reference="Secundário"))[T.Pós graduação]           0.8575      0.064     13.390      0.000       0.732       0.983
C(educacao, Treatment(reference="Secundário"))[T.Superior completo]       0.8431      0.013     63.888      0.000       0.817       0.869
C(educacao, Treatment(reference="Secundário"))[T.Superior incompleto]     0.7723      0.018     42.783      0.000       0.737       0.808
posse_de_imovel[T.True]                                                   0.0834      0.014      6.055      0.000       0.056       0.110
educacao[T.Pós graduação]                                                 0.8575      0.064     13.390      0.000       0.732       0.983
educacao[T.Secundário]                                                    1.5659      0.026     60.148      0.000       1.515       1.617
educacao[T.Superior completo]                                             0.8431      0.013     63.888      0.000       0.817       0.869
educacao[T.Superior incompleto]                                           0.7723      0.018     42.783      0.000       0.737       0.808
qtd_filhos                                                                0.0285      0.019      1.471      0.141      -0.009       0.067
idade                                                                     0.0054      0.001      6.969      0.000       0.004       0.007
tempo_emprego                                                             0.0609      0.001     59.126      0.000       0.059       0.063
qt_pessoas_residencia                                                    -0.0051      0.016     -0.317      0.751      -0.037       0.027
==============================================================================
Omnibus:                        0.831   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.660   Jarque-Bera (JB):                0.816
Skew:                           0.019   Prob(JB):                        0.665
Kurtosis:                       3.009   Cond. No.                     5.02e+18
==================================================================

Considerando a remocao das variaveis educacao, tipo_renda e tipo_residencia, embora o coeficiente de determinacao diminua, podemos interpretar que estamos evitando o over fitting.

In [70]:
y, x = patsy.dmatrices('np.log(renda) ~ qtd_filhos + qt_pessoas_residencia', data=df)
x

DesignMatrix with shape (15000, 3)
  Intercept  qtd_filhos  qt_pessoas_residencia
          1           0                      1
          1           0                      2
          1           0                      2
          1           1                      3
          1           0                      1
          1           0                      2
          1           0                      1
          1           0                      2
          1           0                      2
          1           0                      2
          1           0                      2
          1           0                      2
          1           0                      1
          1           0                      2
          1           0                      2
          1           1                      2
          1           0                      2
          1           0                      1
          1           1                      2
          1           0  

In [72]:
sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     37.50
Date:                Thu, 29 Aug 2024   Prob (F-statistic):           5.66e-17
Time:                        21:04:17   Log-Likelihood:                -19387.
No. Observations:               15000   AIC:                         3.878e+04
Df Residuals:                   14997   BIC:                         3.880e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 7.9951      0.031    255.917      0.000       7.934       8.056
qtd_filhos               -0.0596      0.021     -2.821      0.005      -0.101      -0.018
qt_pessoas_residencia     0.1083      0.017      6.247      0.000       0.074       0.142
==============================================================================
Omnibus:                      318.926   Durbin-Watson:                   2.031
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              360.407
Skew:                           0.323   Prob(JB):                     5.48e-79
Kurtosis:                       3.400   Cond. No.                         14.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Considerando a requisicao do cliente para que sejam removidas as variaveis com p-value menor do que 5%, sobrando apenas as variaveis qtd_filhos e qt_pessoas_residencia, podemos afirmar entao que o modelo piorou com relacao aos anteriores. Avaliando o inverso a seguir, teremos entao:

In [75]:
y, x = patsy.dmatrices('np.log(renda) ~ C(sexo,Treatment(reference="F")) + posse_de_veiculo + C(educacao,Treatment(reference="Secundário")) + posse_de_imovel + educacao + idade + tempo_emprego', data=df)
x

DesignMatrix with shape (12427, 14)
  Columns:
    ['Intercept',
     'C(sexo, Treatment(reference="F"))[T.M]',
     'posse_de_veiculo[T.True]',
     'C(educacao, Treatment(reference="Secundário"))[T.Primário]',
     'C(educacao, Treatment(reference="Secundário"))[T.Pós graduação]',
     'C(educacao, Treatment(reference="Secundário"))[T.Superior completo]',
     'C(educacao, Treatment(reference="Secundário"))[T.Superior incompleto]',
     'posse_de_imovel[T.True]',
     'educacao[T.Pós graduação]',
     'educacao[T.Secundário]',
     'educacao[T.Superior completo]',
     'educacao[T.Superior incompleto]',
     'idade',
     'tempo_emprego']
  Terms:
    'Intercept' (column 0)
    'C(sexo, Treatment(reference="F"))' (column 1)
    'posse_de_veiculo' (column 2)
    'C(educacao, Treatment(reference="Secundário"))' (columns 3:7)
    'posse_de_imovel' (column 7)
    'educacao' (columns 8:12)
    'idade' (column 12)
    'tempo_emprego' (column 13)
  (to view full data, use np.asarray(this_ob

In [77]:
sm.OLS(y,x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.350
Model:                            OLS   Adj. R-squared:                  0.350
Method:                 Least Squares   F-statistic:                     744.4
Date:                Thu, 29 Aug 2024   Prob (F-statistic):               0.00
Time:                        21:06:33   Log-Likelihood:                -13635.
No. Observations:               12427   AIC:                         2.729e+04
Df Residuals:                   12417   BIC:                         2.736e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
=========================================================================================================================================
                                                                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 5.6214      0.033    169.976      0.000       5.557       5.686
C(sexo, Treatment(reference="F"))[T.M]                                    0.7794      0.015     53.520      0.000       0.751       0.808
posse_de_veiculo[T.True]                                                  0.0475      0.014      3.377      0.001       0.020       0.075
C(educacao, Treatment(reference="Secundário"))[T.Primário]                1.5669      0.058     26.874      0.000       1.453       1.681
C(educacao, Treatment(reference="Secundário"))[T.Pós graduação]           0.8629      0.064     13.490      0.000       0.738       0.988
C(educacao, Treatment(reference="Secundário"))[T.Superior completo]       0.8461      0.013     66.232      0.000       0.821       0.871
C(educacao, Treatment(reference="Secundário"))[T.Superior incompleto]     0.7740      0.018     43.542      0.000       0.739       0.809
posse_de_imovel[T.True]                                                   0.0846      0.014      6.152      0.000       0.058       0.112
educacao[T.Pós graduação]                                                 0.8629      0.064     13.490      0.000       0.738       0.988
educacao[T.Secundário]                                                    1.5717      0.025     62.665      0.000       1.522       1.621
educacao[T.Superior completo]                                             0.8461      0.013     66.232      0.000       0.821       0.871
educacao[T.Superior incompleto]                                           0.7740      0.018     43.542      0.000       0.739       0.809
idade                                                                     0.0049      0.001      6.486      0.000       0.003       0.006
tempo_emprego                                                             0.0610      0.001     59.247      0.000       0.059       0.063
==============================================================================
Omnibus:                        0.844   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.656   Jarque-Bera (JB):                0.830
Skew:                           0.020   Prob(JB):                        0.660
Kurtosis:                       3.008   Cond. No.                     5.88e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.51e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""